# 0.0 Imports

In [84]:
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble      import RandomForestRegressor
import seaborn as sns
from matplotlib            import pyplot as plt

# 1.0 Load Data


In [3]:
dscombined = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined.csv' )
dsLIWC = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combinedLIWC.csv' )
dsCohmetrix = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined_CohMetrix.csv' )


# 2.0 Merging data

In [7]:
dsCohmetrix = dsCohmetrix.rename( columns={ 'ID':'id' } )

In [8]:
mergedData = pd.merge(dsCohmetrix, dsLIWC, how='left', on='id')
mergedData = pd.merge(mergedData, dscombined, how='left', on='id')
# df1.merge(df2, how='inner', on='a')

In [9]:
mergedData

,id,cm.DESPC,cm.DESSC,cm.DESWC,cm.DESPL,cm.DESPLd,cm.DESSL,cm.DESSLd,cm.DESWLsy,cm.DESWLsyd,...,liwc.leisure,liwc.home,liwc.money,liwc.relig,liwc.death,liwc.assent,liwc.nonfl,liwc.filler,feedback,classe
0,23,1,1,18,1.0,0.0,17.000000,0.000000,1.888889,0.900254,...,1,0,0,0,0,0,0,0,Ler outros textos sobre o assunto para lher da...,0
1,24,1,2,24,2.0,0.0,10.500000,0.500000,1.916667,1.100066,...,1,0,1,0,0,0,0,0,Procure ler mais; para escrever mais acerca do...,0
2,35,1,2,31,2.0,0.0,12.500000,0.500000,1.741935,0.964978,...,2,0,0,0,0,0,0,0,Texto bom; precisa apenas ler mais um pouco; p...,0
3,37,1,1,7,1.0,0.0,6.000000,0.000000,1.857143,1.069045,...,0,0,0,0,0,0,0,0,Ter cuidado com copias da internet.,0
4,68,1,4,32,4.0,0.0,6.500000,3.640055,1.937500,1.268413,...,0,0,1,0,0,2,0,0,Carssimo; o trabalho precisa de uma sistematiz...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13809,1,2,28,2.0,0.0,12.000000,11.000000,2.250000,1.404358,...,0,0,1,0,0,0,1,0,Rosivalda; timo texto; suas reflexes comparati...,1
996,13810,1,2,66,2.0,0.0,29.500000,27.500000,1.969697,1.149845,...,2,0,1,0,0,1,0,0,Ruth; bom texto sobre o filme e sobre as suas ...,1
997,13811,1,2,37,2.0,0.0,16.500000,15.500000,2.027027,1.213042,...,2,2,0,0,0,1,2,0,Silmara; excelente texto e leitura do filme co...,1
998,13813,1,3,28,3.0,0.0,7.666667,10.143416,1.964286,1.170063,...,2,0,0,1,0,0,1,0,Sueli; boa leitura do filme e da realidade; su...,1


## 2.1 Check Nan

In [68]:
mergedData.isna().sum().sort_values()


id                0
liwc.ipron        0
liwc.article      0
liwc.verb         0
liwc.auxverb      0
               ... 
cm.RDL2         165
cm.CRFCWO1      165
cm.CRFCWO1d     165
cm.LSASSpd      362
cm.LSASSp       362
Length: 161, dtype: int64

## 2.2 Fillout NAn

In [71]:
mergedData = mergedData.dropna(axis=1, how='any', inplace=False) # axis=1 => columns

In [72]:
mergedData.isna().sum().sort_values()

id              0
liwc.article    0
liwc.verb       0
liwc.auxverb    0
liwc.past       0
               ..
cm.SMCAUSr      0
cm.SMCAUSlsa    0
cm.SMCAUSwn     0
cm.CNCComp      0
classe          0
Length: 155, dtype: int64

# 3.0 Data Preparation



## 3.1 Normalization

In [10]:
# no analyzed variable looks like a normal distribution

## 3.2 Rescaling

In [11]:
mergedData.dtypes

id               int64
cm.DESPC         int64
cm.DESSC         int64
cm.DESWC         int64
cm.DESPL       float64
                ...   
liwc.assent      int64
liwc.nonfl       int64
liwc.filler      int64
feedback        object
classe           int64
Length: 161, dtype: object

In [12]:
visu = mergedData.select_dtypes( exclude=['int64', 'float64','int32','UInt32'] )
print(visu)

                                              feedback
0    Ler outros textos sobre o assunto para lher da...
1    Procure ler mais; para escrever mais acerca do...
2    Texto bom; precisa apenas ler mais um pouco; p...
3                  Ter cuidado com copias da internet.
4    Carssimo; o trabalho precisa de uma sistematiz...
..                                                 ...
995  Rosivalda; timo texto; suas reflexes comparati...
996  Ruth; bom texto sobre o filme e sobre as suas ...
997  Silmara; excelente texto e leitura do filme co...
998  Sueli; boa leitura do filme e da realidade; su...
999  Taciana; boa leitura do filme e da realidade; ...

[1000 rows x 1 columns]


In [13]:
# thus, due to feedback be a no-analyzed variable, merged dataset doesn't need rescaling
# so, the other variables doesnt need a changes, for now, because the outliers problem will be solved on cohmetrix software  

## 3.3 Transformation


In [14]:
# once time again, transformation is not necessary, due to mergedData only have feedback that is a categorical variable and the response variable is only 0 and 1

# 4.0 EDA - Correlation of numAtttributes

In [78]:
num_attributes = mergedData.select_dtypes( include=['int64', 'float64'] )

In [ ]:
correlation = num_attributes.corr( method='pearson' )
plt.figure( figsize=( 25, 19 ) )
sns.heatmap( correlation, annot=True );

# 5.0 Feature Selection


## 4.1 Split dataframe into training and test dataset

In [73]:
# training dataset
X_train = mergedData[mergedData['id']<12800] # this id split dataset in 3/4 proportion
y_train = X_train['classe']

# test dataset 
X_test = mergedData[mergedData['id']>=12800] # this id split dataset in 3/4 proportion
y_test = X_test['classe']


## 4.2 Boruta as Feature Selector

In [16]:
# training and test dataset for Boruta
X_train_n = X_train.drop( ['classe', 'feedback'], axis=1 ).values 
y_train_n = y_train.values.ravel() 

# define RandomForestRegressor
rf = RandomForestRegressor( n_jobs=-1 )

# define Boruta
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n )


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_train.drop( ['classe', 'feedback', 'id'], axis=1 ).select_dtypes( include=['float64'] )